This is a SageMath notebook.
Peter Luschny
2024/01/25

We study the compositions of n where the first part is the largest part and the last part is not 1. (A368279 and A369492.)

In [1]:
def _A368279Compositions(n: int, verbose=False):
    """
    Generates all interpart restricted compositions of n (n >= 1) with 
    first part equal to the largest part and the last part not 1.
    An adaption of Kelleher's algorithm. See Kelleher 2006, 'Encoding
    partitions as ascending compositions' chapters 3 and 4 for details.
    https://jeromekelleher.net/generating-integer-partitions.html
    """
    count = 0
    a = [0 for i in range(n + 1)]
    k = 1
    a[0] = 1
    a[1] = n - 1
    while k != 0:
        x = a[k - 1] + 1
        y = a[k] - 1
        k -= 1
        while 1 <= y:
            a[k] = x
            x = 1
            y -= x
            k += 1
        a[k] = x + y
        if a[k] != 1:
            if all(part <= a[0] for part in a[:k + 1]):
                yield a[:k + 1]
                count += 1
    if verbose:
        print(f"The number of A368279-compositions of {n} is {count}.")

Let's first look at the corner cases.<br> 
For n = 1, there are no A368279-compositions, 
(since [1] is the only composition of 1 but we exclude compositions ending with 1), and the algorithm correctly terminates without returning a composition. In this case it is convenient to let the function return [ ].<br> 
For n = 0, the algorithm is not defined. 

In [2]:
def A368279Compositions(n: int, verbose=False):
    if n == 1: return [[]]
    return _A368279Compositions(n, verbose)

In [3]:
for p in A368279Compositions(6): 
    print(p)

[2, 1, 1, 2]
[2, 2, 2]
[3, 1, 2]
[3, 3]
[4, 2]
[6]


The compositions for a fixed n are sorted by ascending first part. Now we want to reverse this and list them by falling first part. The reason is that this way we later get increasing values when representing the compositions as binary words.

In [4]:
def A368279CompositionsInReverseOrder(n):
    stack = [] 
    for c in A368279Compositions(n): 
        stack.append(c)
    return reversed(stack) 

In [5]:
for c in A368279CompositionsInReverseOrder(6): 
    print(c)

[6]
[4, 2]
[3, 3]
[3, 1, 2]
[2, 2, 2]
[2, 1, 1, 2]


Concat the compostions in a single list.

In [6]:
def A368279CompositionsListInReverseOrder(size):
    total = []
    for n in range(1, size):
        total.extend(A368279CompositionsInReverseOrder(n)) 
    return total  

In [7]:
for c in A368279CompositionsListInReverseOrder(7):  
    print(c)

[]
[2]
[3]
[4]
[2, 2]
[5]
[3, 2]
[2, 1, 2]
[6]
[4, 2]
[3, 3]
[3, 1, 2]
[2, 2, 2]
[2, 1, 1, 2]


In [8]:
# Integer compositions -> Binary words
def binword(w):
    return Words([0, 1])(w.to_code()) 

# Integer compositions -> Dyck paths
def dyckpath(c):
    return DyckWord(sum([[1]*a + [0]*a for a in c], []))

Combine the four representations in a table. <br>
Encoded as an integer, a binary string, a dyckpath (brackets), and a composition.

In [9]:
def A368279CompositionsMaps(size):
    for c in A368279CompositionsListInReverseOrder(size):
        bw = str(binword(Composition(c)))
        dyck = str(dyckpath(c))
        dec = int(bw, 2)
        print(f"{dec:3d} | {bw.ljust(7)} | {dyck.ljust(14)} | {c}")

<p style="color:brown; font-size:18px">The Main Table<p>

In [10]:
A368279CompositionsMaps(8)

  0 | 0       |                | []
  2 | 10      | (())           | [2]
  4 | 100     | ((()))         | [3]
  8 | 1000    | (((())))       | [4]
 10 | 1010    | (())(())       | [2, 2]
 16 | 10000   | ((((()))))     | [5]
 18 | 10010   | ((()))(())     | [3, 2]
 22 | 10110   | (())()(())     | [2, 1, 2]
 32 | 100000  | (((((())))))   | [6]
 34 | 100010  | (((())))(())   | [4, 2]
 36 | 100100  | ((()))((()))   | [3, 3]
 38 | 100110  | ((()))()(())   | [3, 1, 2]
 42 | 101010  | (())(())(())   | [2, 2, 2]
 46 | 101110  | (())()()(())   | [2, 1, 1, 2]
 64 | 1000000 | ((((((())))))) | [7]
 66 | 1000010 | ((((()))))(()) | [5, 2]
 68 | 1000100 | (((())))((())) | [4, 3]
 70 | 1000110 | (((())))()(()) | [4, 1, 2]
 74 | 1001010 | ((()))(())(()) | [3, 2, 2]
 76 | 1001100 | ((()))()((())) | [3, 1, 3]
 78 | 1001110 | ((()))()()(()) | [3, 1, 1, 2]
 86 | 1010110 | (())(())()(()) | [2, 2, 1, 2]
 90 | 1011010 | (())()(())(()) | [2, 1, 2, 2]
 94 | 1011110 | (())()()()(()) | [2, 1, 1, 1, 2]


There are other representations, but less illuminating. 
We have the following interpretation of the Dyck paths plotted below:
"A group of elephants moves from right to left.
The boss goes first and minors are placed in
the middle and are not allowed to run at the end."

In [11]:
C = A368279CompositionsInReverseOrder(7)
for c in C: 
    dyck = dyckpath(c)
    dyck.pretty_print(type="NE-SE")
    bit = dyck.to_binary_tree()
    print(bit)
    print(bit.as_ordered_tree())
    print(c); print()

      /\      
     /  \     
    /    \    
   /      \   
  /        \  
 /          \ 
/            \
[[[[[[[., .], .], .], .], .], .], .]
[[[[[[[[], []], []], []], []], []], []], []]
[7]

    /\        
   /  \       
  /    \      
 /      \  /\ 
/        \/  \
[[[[[., .], .], .], .], [[., .], .]]
[[[[[[], []], []], []], []], [[[], []], []]]
[5, 2]

   /\         
  /  \    /\  
 /    \  /  \ 
/      \/    \
[[[[., .], .], .], [[[., .], .], .]]
[[[[[], []], []], []], [[[[], []], []], []]]
[4, 3]

   /\         
  /  \        
 /    \    /\ 
/      \/\/  \
[[[[., .], .], .], [., [[., .], .]]]
[[[[[], []], []], []], [[], [[[], []], []]]]
[4, 1, 2]

  /\          
 /  \  /\  /\ 
/    \/  \/  \
[[[., .], .], [[., .], [[., .], .]]]
[[[[], []], []], [[[], []], [[[], []], []]]]
[3, 2, 2]

  /\      /\  
 /  \    /  \ 
/    \/\/    \
[[[., .], .], [., [[[., .], .], .]]]
[[[[], []], []], [[], [[[[], []], []], []]]]
[3, 1, 3]

  /\          
 /  \      /\ 
/    \/\/\/  \
[[[., .], .], [., [

The compact format: the bitstring of a A368279-composition in decimal representation.

In [12]:
def EncodeA368279Compositions(size):
    total = []
    for n in range(1, size):
        stack = [] 
        for c in A368279Compositions(n): 
            bw = binword(Composition(c))
            stack.append(int(str(bw), 2))
        total.extend(reversed(stack)) 
    return total

In [13]:
EncodeA368279List = EncodeA368279Compositions(10)
print(EncodeA368279List)

[0, 2, 4, 8, 10, 16, 18, 22, 32, 34, 36, 38, 42, 46, 64, 66, 68, 70, 74, 76, 78, 86, 90, 94, 128, 130, 132, 134, 136, 138, 140, 142, 146, 148, 150, 154, 156, 158, 170, 174, 182, 186, 190, 256, 258, 260, 262, 264, 266, 268, 270, 274, 276, 278, 280, 282, 284, 286, 292, 294, 298, 300, 302, 306, 308, 310, 314, 316, 318, 342, 346, 350, 362, 366, 374, 378, 382]


This is now OEIS A369492.

In [14]:
for dec in EncodeA368279List:
    print(f"{dec:3d} | {dec:b}")

  0 | 0
  2 | 10
  4 | 100
  8 | 1000
 10 | 1010
 16 | 10000
 18 | 10010
 22 | 10110
 32 | 100000
 34 | 100010
 36 | 100100
 38 | 100110
 42 | 101010
 46 | 101110
 64 | 1000000
 66 | 1000010
 68 | 1000100
 70 | 1000110
 74 | 1001010
 76 | 1001100
 78 | 1001110
 86 | 1010110
 90 | 1011010
 94 | 1011110
128 | 10000000
130 | 10000010
132 | 10000100
134 | 10000110
136 | 10001000
138 | 10001010
140 | 10001100
142 | 10001110
146 | 10010010
148 | 10010100
150 | 10010110
154 | 10011010
156 | 10011100
158 | 10011110
170 | 10101010
174 | 10101110
182 | 10110110
186 | 10111010
190 | 10111110
256 | 100000000
258 | 100000010
260 | 100000100
262 | 100000110
264 | 100001000
266 | 100001010
268 | 100001100
270 | 100001110
274 | 100010010
276 | 100010100
278 | 100010110
280 | 100011000
282 | 100011010
284 | 100011100
286 | 100011110
292 | 100100100
294 | 100100110
298 | 100101010
300 | 100101100
302 | 100101110
306 | 100110010
308 | 100110100
310 | 100110110
314 | 100111010
316 | 100111100
318 | 100111

Note that this sequence can be written as a table with row n having length A368279(n) (for n >= 1). How the row length can be calculated is shown in the appendix. <br> 
   
   0;<br>
   2;<br>
   4;<br>
   8, 10;<br>
  16, 18, 22;<br>
  32, 34, 36, 38, 42, 46;<br>
  64, 66, 68, 70, 74, 76, 78, 86, 90, 94;<br>
  ...;

<p style="color:brown; font-size:18px">APPENDIX (Cardinality)</p>

In [15]:
@cached_function
def F(k, n): 
    return sum(F(k, n-j) for j in range(1, min(k, n))) if n > 1 else n
def a(n): return sum(F(k+1, n+1-k) - F(k+1, n-k) for k in range(n+1))

In [16]:
print([a(n) for n in (0..12)])

[1, 0, 1, 1, 2, 3, 6, 10, 19, 34, 63, 116, 216]


In [17]:
def C(n): 
    return sum(Compositions(n, max_part=k, inner=[k]).cardinality()
           for k in (0..n))
def a(n): return C(n) - C(n-1) if n > 1 else 1 - n

In [18]:
print([a(n) for n in (0..12)])

[1, 0, 1, 1, 2, 3, 6, 10, 19, 34, 63, 116, 216]


Generating function for A368279.

$$ (1 - x) \sum_{j=0}^\infty \frac{x^j} {1 - \sum_{k=1}^j x^k} $$

In [19]:
def A368279List(prec):
    P.<x> = PowerSeriesRing(QQ, prec)
    return P((1 - x) * sum(x^j / (1 - sum(x^k for k in range(1, j+1))) 
                       for j in range(0, prec+2))).list()

In [20]:
print(A368279List(12))

[1, 0, 1, 1, 2, 3, 6, 10, 19, 34, 63, 116, 216]


Finally, we remark that A079500 - A368279 = A007059 and that A368279 are the row sums of A368579.